<a href="https://colab.research.google.com/github/solobala/RMSL_9/blob/main/RS_lecture_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

In [ ]:
!wget https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
!unzip master.zip

--2023-06-27 16:40:35--  https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master [following]
--2023-06-27 16:40:35--  https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   1.06M  4.42MB/s    in 0.2s    

2023-06-27 16:40:36 (4.42 MB/s) - ‘master.zip’ saved [1111929]

Archive:  master.zip
dfe2a910caf170a1f0fd2174867169ce737c9dc7
   creating: netology-recsys-master/
   creating: netolo

In [ ]:
prefix = 'netology-recsys-master/lecture-1'

movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [ ]:
num_users = movies_with_ratings.userId.unique().shape[0]

In [ ]:
num_users

610

In [ ]:
movie_vector = {}

for movie, group in tqdm_notebook(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users)

    for i in range(len(group.userId.values)):
        u = group.userId.values[i]  # user index
        r = group.rating.values[i]  # which rating user u gave to movie
        movie_vector[movie][int(u - 1)] = r

<ipython-input-10-80f6ac1d2431>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for movie, group in tqdm_notebook(movies_with_ratings.groupby('title')):


  0%|          | 0/9719 [00:00<?, ?it/s]

In [ ]:
movie_vector['Toy Story (1995)'].shape

(610,)

In [ ]:
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard, correlation

In [ ]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm_notebook(movie_vector.keys()):
    if key == my_fav_film:
        continue

    titles.append(key)
    distances.append(correlation(movie_vector[my_fav_film], movie_vector[key]))

best_indexes = np.argsort(distances)[:10]
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m)

<ipython-input-13-0a8d6f3d4ceb>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for key in tqdm_notebook(movie_vector.keys()):


  0%|          | 0/9719 [00:00<?, ?it/s]

('Matrix, The (1999)', 0.461785637779797)
('Memento (2000)', 0.46277036445237296)
('American History X (1998)', 0.4698485700562477)
('Kill Bill: Vol. 1 (2003)', 0.48208819852069573)
('Kill Bill: Vol. 2 (2004)', 0.49550594983117513)
('Inception (2010)', 0.5253368759258412)
('Departed, The (2006)', 0.5312318420920308)
('Eternal Sunshine of the Spotless Mind (2004)', 0.5323396310804864)
('Reservoir Dogs (1992)', 0.5358995135923637)
('Lord of the Rings: The Fellowship of the Ring, The (2001)', 0.5396526638176289)


In [ ]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm_notebook(movie_vector.keys()):
    if key == my_fav_film:
        continue

    titles.append(key)
    distances.append(jaccard(movie_vector[my_fav_film], movie_vector[key]))

best_indexes = np.argsort(distances)[:10]
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m[0], f"\tjaccard: {m[1]:.3f}")

<ipython-input-14-520f6edfb830>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for key in tqdm_notebook(movie_vector.keys()):


  0%|          | 0/9719 [00:00<?, ?it/s]

Matrix, The (1999) 	jaccard: 0.782
Memento (2000) 	jaccard: 0.791
Pulp Fiction (1994) 	jaccard: 0.820
Lord of the Rings: The Fellowship of the Ring, The (2001) 	jaccard: 0.824
Seven (a.k.a. Se7en) (1995) 	jaccard: 0.832
American History X (1998) 	jaccard: 0.833
American Beauty (1999) 	jaccard: 0.840
Lord of the Rings: The Two Towers, The (2002) 	jaccard: 0.849
Lord of the Rings: The Return of the King, The (2003) 	jaccard: 0.850
Reservoir Dogs (1992) 	jaccard: 0.851


In [ ]:
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm_notebook(movie_vector.keys()):
    if key == my_fav_film:
        continue

    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key]))

best_indexes = np.argsort(distances)[:10]
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m[0], f"\tCosine distance: {m[1]:.3f}")

<ipython-input-15-258e00f3058d>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for key in tqdm_notebook(movie_vector.keys()):


  0%|          | 0/9719 [00:00<?, ?it/s]

Matrix, The (1999) 	Cosine distance: 0.286
Memento (2000) 	Cosine distance: 0.330
American History X (1998) 	Cosine distance: 0.351
Kill Bill: Vol. 1 (2003) 	Cosine distance: 0.360
Lord of the Rings: The Fellowship of the Ring, The (2001) 	Cosine distance: 0.364
American Beauty (1999) 	Cosine distance: 0.374
Pulp Fiction (1994) 	Cosine distance: 0.377
Lord of the Rings: The Return of the King, The (2003) 	Cosine distance: 0.378
Inception (2010) 	Cosine distance: 0.385
Kill Bill: Vol. 2 (2004) 	Cosine distance: 0.386
